In this notebook the original train and test datasets are analysed and encoded so that they can be used to train machine learning regression models

In [92]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.impute import KNNImputer

# 1. Load data

In [3]:
train_dataset = pd.read_csv('train.csv', sep = ',')

In [4]:
test_dataset = pd.read_csv('test.csv', sep = ',')

In the test dataset, the SalePrice column is added so both dataframes have the same columns and can be concatenated. It is given NaN values.

In [5]:
test_dataset['SalePrice'] = np.nan

Both dataframes are concatenated so that the encoding of the both datasets is done at the same time and equally

In [6]:
traintest_dataset = pd.concat([train_dataset, test_dataset])

In [7]:
traintest_dataset.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive

In [8]:
traintest_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2919 entries, 0 to 1458
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             2919 non-null   int64  
 1   MSSubClass     2919 non-null   int64  
 2   MSZoning       2915 non-null   object 
 3   LotFrontage    2433 non-null   float64
 4   LotArea        2919 non-null   int64  
 5   Street         2919 non-null   object 
 6   Alley          198 non-null    object 
 7   LotShape       2919 non-null   object 
 8   LandContour    2919 non-null   object 
 9   Utilities      2917 non-null   object 
 10  LotConfig      2919 non-null   object 
 11  LandSlope      2919 non-null   object 
 12  Neighborhood   2919 non-null   object 
 13  Condition1     2919 non-null   object 
 14  Condition2     2919 non-null   object 
 15  BldgType       2919 non-null   object 
 16  HouseStyle     2919 non-null   object 
 17  OverallQual    2919 non-null   int64  
 18  OverallC

In [9]:
traintest_dataset.isna().sum()

Id                  0
MSSubClass          0
MSZoning            4
LotFrontage       486
LotArea             0
                 ... 
MoSold              0
YrSold              0
SaleType            1
SaleCondition       0
SalePrice        1459
Length: 81, dtype: int64

There is a significant amount of missing data in the resulting dataset, after taking a look at the variables description, most of this missing values are filled in the most appropiate way possible

Note: All of this substitutions can be done together by selecting a list of variables from traintest_dataset instead of going 1 by 1. It is done this way for code clarity. (It is arguable if the other way is better)

In [10]:
traintest_dataset['Alley'] = traintest_dataset['Alley'].fillna('No')
traintest_dataset['BsmtFinType1'] = traintest_dataset['BsmtFinType1'].fillna('No')
traintest_dataset['BsmtFinType2'] = traintest_dataset['BsmtFinType2'].fillna('No')
traintest_dataset['GarageType'] = traintest_dataset['GarageType'].fillna('No')
traintest_dataset['GarageFinish'] = traintest_dataset['GarageFinish'].fillna('No')
traintest_dataset['Fence'] = traintest_dataset['Fence'].fillna('No')
traintest_dataset['MiscFeature'] = traintest_dataset['MiscFeature'].fillna('No')

In [11]:
traintest_dataset['BsmtQual'] = traintest_dataset['BsmtQual'].fillna('No')
traintest_dataset['BsmtCond'] = traintest_dataset['BsmtCond'].fillna('No')
traintest_dataset['BsmtExposure'] = traintest_dataset['BsmtExposure'].fillna('Na')
traintest_dataset['FireplaceQu'] = traintest_dataset['FireplaceQu'].fillna('No')
traintest_dataset['GarageQual'] = traintest_dataset['GarageQual'].fillna('No')
traintest_dataset['GarageCond'] = traintest_dataset['GarageCond'].fillna('No')
traintest_dataset['PoolQC'] = traintest_dataset['PoolQC'].fillna('No')

In [12]:
traintest_dataset['LotFrontage'] = traintest_dataset['LotFrontage'].fillna(0)
traintest_dataset['GarageYrBlt'] = traintest_dataset['GarageYrBlt'].fillna(0)
traintest_dataset['MasVnrArea'] = traintest_dataset['MasVnrArea'].fillna(0)

In [14]:
traintest_dataset[['BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath']] = traintest_dataset[['BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF',
                                                                                                                                 'TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath']].fillna(0)

For the case of the 'MasVnrType' column, not all missing values should be set to None, only those in which 'MasVnrArea' takes value 0

In [26]:
tmp = traintest_dataset[traintest_dataset['MasVnrType'].isna()]
row_index = tmp[tmp['MasVnrArea'] != 0.0].index[0]

In [29]:
traintest_dataset[traintest_dataset['MasVnrType'].isna()][['MasVnrType', 'MasVnrArea']]

,MasVnrType,MasVnrArea
1150,NaN,198.0


In [28]:
traintest_dataset['MasVnrType'] = traintest_dataset.apply(lambda x: 'None' if pd.isna(x['MasVnrType']) and x.name != row_index else x['MasVnrType'], axis=1)

For the rest of missing values knowing which values to impute is not obvious.\
For this reason, k-Nearest Neighbours is going to be used for this data imputation. This will be done after the dataset is encoded

# 2. Encoding

From examining the data and looking at the variables description of the dataset, it can be seen that this dataset contains a significant amount of categorical data. In order for this data to be used to train a machine learning model these variables need to be encoded. For this encoding 2 different approaches will be followed depending on the nature of the data. 

For those variables whose categories are ordered, such as ratings, they will be encoded in a way that the ordering is preserved, while in the case of categories that are unordered a One Hot Encoder will be used to binarize the data in a way that that there is no partial ordering between any of the categories.

## 2.1 One Hot Encoding

In [30]:
#Create the list of variables that need to be one hot encoded
ohenc_list = ['MSSubClass', 'MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
              'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'Foundation', 'Heating', 'Electrical',
              'Functional', 'GarageType', 'Fence', 'MiscFeature', 'SaleType', 'SaleCondition']

In [31]:
ohenc = OneHotEncoder(sparse=False)

In [32]:
ohenc_dataset = pd.DataFrame(ohenc.fit_transform(traintest_dataset[ohenc_list]), index = traintest_dataset['Id'], columns = list(ohenc.get_feature_names_out()))

## 2.2. Ordinal Encoding

In [33]:
rating_list = ['No', 'Po', 'Fa', 'TA', 'Gd', 'Ex']
exp_list = ['Na', 'No', 'Mn', 'Av', 'Gd']	
fin_list = ['No', 'Unf', 'LwQ', 'Rec', 'BLQ', 'ALQ', 'GLQ']
yesno_list = ['N', 'Y']  
gar_fin_list = ['No', 'Unf', 'RFn', 'Fin']
pav_dr_list = ['N', 'P', 'Y']
ordenc = OrdinalEncoder(categories = [rating_list, rating_list, rating_list, rating_list, exp_list, fin_list, fin_list, rating_list, yesno_list, rating_list,
                                      rating_list, gar_fin_list, rating_list, rating_list, pav_dr_list, rating_list], handle_unknown = 'use_encoded_value',
                                      unknown_value = np.nan)

In [34]:
enc_var_list = ['ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'HeatingQC', 'CentralAir', 'KitchenQual',
                'FireplaceQu', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC']

In [35]:
tmp = ordenc.fit_transform(traintest_dataset[enc_var_list])

In [36]:
ordenc_dataset = pd.DataFrame(tmp, index = traintest_dataset['Id'], columns = enc_var_list)

In [37]:
ordenc_dataset

,ExterQual,ExterCond,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,HeatingQC,CentralAir,KitchenQual,FireplaceQu,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC
Id,,,,,,,,,,,,,,,,
1,4.0,3.0,4.0,3.0,1.0,6.0,1.0,5.0,1.0,4.0,0.0,2.0,3.0,3.0,2.0,0.0
2,3.0,3.0,4.0,3.0,4.0,5.0,1.0,5.0,1.0,3.0,3.0,2.0,3.0,3.0,2.0,0.0
3,4.0,3.0,4.0,3.0,2.0,6.0,1.0,5.0,1.0,4.0,3.0,2.0,3.0,3.0,2.0,0.0
4,3.0,3.0,3.0,4.0,1.0,5.0,1.0,4.0,1.0,4.0,4.0,1.0,3.0,3.0,2.0,0.0
5,4.0,3.0,4.0,3.0,3.0,6.0,1.0,5.0,1.0,4.0,3.0,2.0,3.0,3.0,2.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2915,3.0,3.0,3.0,3.0,1.0,1.0,1.0,4.0,1.0,3.0,0.0,0.0,0.0,0.0,2.0,0.0
2916,3.0,3.0,3.0,3.0,1.0,3.0,1.0,3.0,1.0,3.0,0.0,1.0,3.0,3.0,2.0,0.0
2917,3.0,3.0,3.0,3.0,1.0,5.0,1.0,5.0,1.0,3.0,3.0,1.0,3.0,3.0,2.0,0.0


# 3. Construct final dataset

The pandas dataframes with the One Hot Encoded, the Ordinal Encoded and the non-encoded data must be merged together to form the complete dataset

In [38]:
non_enc_data = traintest_dataset.drop(columns = ohenc_list + enc_var_list)

In [39]:
non_enc_data.set_index('Id', inplace = True)

In [40]:
non_enc_data

,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,65.0,8450,7,5,2003,2003,196.0,706.0,0.0,150.0,...,0,61,0,0,0,0,0,2,2008,208500.0
2,80.0,9600,6,8,1976,1976,0.0,978.0,0.0,284.0,...,298,0,0,0,0,0,0,5,2007,181500.0
3,68.0,11250,7,5,2001,2002,162.0,486.0,0.0,434.0,...,0,42,0,0,0,0,0,9,2008,223500.0
4,60.0,9550,7,5,1915,1970,0.0,216.0,0.0,540.0,...,0,35,272,0,0,0,0,2,2006,140000.0
5,84.0,14260,8,5,2000,2000,350.0,655.0,0.0,490.0,...,192,84,0,0,0,0,0,12,2008,250000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2915,21.0,1936,4,7,1970,1970,0.0,0.0,0.0,546.0,...,0,0,0,0,0,0,0,6,2006,NaN
2916,21.0,1894,4,5,1970,1970,0.0,252.0,0.0,294.0,...,0,24,0,0,0,0,0,4,2006,NaN
2917,160.0,20000,5,7,1960,1996,0.0,1224.0,0.0,0.0,...,474,0,0,0,0,0,0,9,2006,NaN


In [41]:
tmp = pd.merge(ohenc_dataset, ordenc_dataset, how = 'inner', on = 'Id')
complete_dataset = pd.merge(non_enc_data, tmp, how = 'inner', on = 'Id')

In [42]:
complete_dataset = complete_dataset.astype('float64')

In [43]:
complete_dataset

,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,...,BsmtFinType2,HeatingQC,CentralAir,KitchenQual,FireplaceQu,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC
Id,,,,,,,,,,,,,,,,,,,,,
1,65.0,8450.0,7.0,5.0,2003.0,2003.0,196.0,706.0,0.0,150.0,...,1.0,5.0,1.0,4.0,0.0,2.0,3.0,3.0,2.0,0.0
2,80.0,9600.0,6.0,8.0,1976.0,1976.0,0.0,978.0,0.0,284.0,...,1.0,5.0,1.0,3.0,3.0,2.0,3.0,3.0,2.0,0.0
3,68.0,11250.0,7.0,5.0,2001.0,2002.0,162.0,486.0,0.0,434.0,...,1.0,5.0,1.0,4.0,3.0,2.0,3.0,3.0,2.0,0.0
4,60.0,9550.0,7.0,5.0,1915.0,1970.0,0.0,216.0,0.0,540.0,...,1.0,4.0,1.0,4.0,4.0,1.0,3.0,3.0,2.0,0.0
5,84.0,14260.0,8.0,5.0,2000.0,2000.0,350.0,655.0,0.0,490.0,...,1.0,5.0,1.0,4.0,3.0,2.0,3.0,3.0,2.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2915,21.0,1936.0,4.0,7.0,1970.0,1970.0,0.0,0.0,0.0,546.0,...,1.0,4.0,1.0,3.0,0.0,0.0,0.0,0.0,2.0,0.0
2916,21.0,1894.0,4.0,5.0,1970.0,1970.0,0.0,252.0,0.0,294.0,...,1.0,3.0,1.0,3.0,0.0,1.0,3.0,3.0,2.0,0.0
2917,160.0,20000.0,5.0,7.0,1960.0,1996.0,0.0,1224.0,0.0,0.0,...,1.0,5.0,1.0,3.0,3.0,1.0,3.0,3.0,2.0,0.0


There is one issue with using scikit-learn's OneHotEncoder to encode the categorical values and that issue is that when it encounters NaN values, it treates them as an extra category of that variable, creating an extra column.\
To fix this issue, in the following code this extra NaN columns will be eliminated and the values of the rest of the columns of that one hot encoded category will be changed to NaN for that row.

In [44]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(complete_dataset.isna().sum())

LotFrontage                 0
LotArea                     0
OverallQual                 0
OverallCond                 0
YearBuilt                   0
YearRemodAdd                0
MasVnrArea                  0
BsmtFinSF1                  0
BsmtFinSF2                  0
BsmtUnfSF                   0
TotalBsmtSF                 0
1stFlrSF                    0
2ndFlrSF                    0
LowQualFinSF                0
GrLivArea                   0
BsmtFullBath                0
BsmtHalfBath                0
FullBath                    0
HalfBath                    0
BedroomAbvGr                0
KitchenAbvGr                0
TotRmsAbvGrd                0
Fireplaces                  0
GarageYrBlt                 0
GarageCars                  1
GarageArea                  1
WoodDeckSF                  0
OpenPorchSF                 0
EnclosedPorch               0
3SsnPorch                   0
ScreenPorch                 0
PoolArea                    0
MiscVal                     0
MoSold    

MS_Zoning

In [45]:
MSZoning_nan_indexes = list(complete_dataset[complete_dataset['MSZoning_nan'] == 1].index.values)

In [46]:
cols_to_replace = ['MSZoning_C (all)', 'MSZoning_FV', 'MSZoning_RH', 'MSZoning_RL', 'MSZoning_RM']
# Replace values at specified positions with NaN
for row_name in MSZoning_nan_indexes:
    row_idx = complete_dataset.index.get_loc(row_name)
    for col_name in cols_to_replace:
        col_idx = complete_dataset.columns.get_loc(col_name)
        complete_dataset.iloc[row_idx, col_idx] = np.nan

In [47]:
complete_dataset[complete_dataset.index.isin(MSZoning_nan_indexes)][['MSZoning_C (all)', 'MSZoning_FV', 'MSZoning_RH', 'MSZoning_RL', 'MSZoning_RM']]

,MSZoning_C (all),MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM
Id,,,,,
1916,NaN,NaN,NaN,NaN,NaN
2217,NaN,NaN,NaN,NaN,NaN
2251,NaN,NaN,NaN,NaN,NaN
2905,NaN,NaN,NaN,NaN,NaN


In [48]:
complete_dataset.drop(columns = ['MSZoning_nan'], inplace = True)

Utilities

In [50]:
Utilities_nan_indexes = list(complete_dataset[complete_dataset['Utilities_nan'] == 1].index.values)

In [51]:
cols_to_replace = ['Utilities_AllPub', 'Utilities_NoSeWa']
# Replace values at specified positions with NaN
for row_name in Utilities_nan_indexes:
    row_idx = complete_dataset.index.get_loc(row_name)
    for col_name in cols_to_replace:
        col_idx = complete_dataset.columns.get_loc(col_name)
        complete_dataset.iloc[row_idx, col_idx] = np.nan

In [52]:
complete_dataset[complete_dataset.index.isin(Utilities_nan_indexes)][['Utilities_AllPub', 'Utilities_NoSeWa']]

,Utilities_AllPub,Utilities_NoSeWa
Id,,
1916,NaN,NaN
1946,NaN,NaN


In [53]:
complete_dataset.drop(columns = ['Utilities_nan'], inplace = True)

Exterior1st

In [54]:
Exterior1st_nan_indexes = list(complete_dataset[complete_dataset['Exterior1st_nan'] == 1].index.values)

In [164]:
Exterior1st_nan_indexes

[2152]

In [55]:
cols_to_replace = ['Exterior1st_AsbShng', 'Exterior1st_AsphShn', 'Exterior1st_BrkComm', 'Exterior1st_BrkFace', 'Exterior1st_CBlock', 'Exterior1st_CemntBd', 'Exterior1st_HdBoard',
                    'Exterior1st_ImStucc', 'Exterior1st_MetalSd', 'Exterior1st_Plywood', 'Exterior1st_Stone', 'Exterior1st_Stucco', 'Exterior1st_VinylSd', 'Exterior1st_Wd Sdng',
                    'Exterior1st_WdShing']
# Replace values at specified positions with NaN
for row_name in Exterior1st_nan_indexes:
    row_idx = complete_dataset.index.get_loc(row_name)
    for col_name in cols_to_replace:
        col_idx = complete_dataset.columns.get_loc(col_name)
        complete_dataset.iloc[row_idx, col_idx] = np.nan

In [56]:
complete_dataset[complete_dataset.index.isin(Exterior1st_nan_indexes)][['Exterior1st_AsbShng', 'Exterior1st_AsphShn', 'Exterior1st_BrkComm', 'Exterior1st_BrkFace', 'Exterior1st_CBlock',
                                                                         'Exterior1st_CemntBd', 'Exterior1st_HdBoard', 'Exterior1st_ImStucc', 'Exterior1st_MetalSd', 'Exterior1st_Plywood',
                                                                           'Exterior1st_Stone', 'Exterior1st_Stucco', 'Exterior1st_VinylSd', 'Exterior1st_Wd Sdng', 'Exterior1st_WdShing']]

,Exterior1st_AsbShng,Exterior1st_AsphShn,Exterior1st_BrkComm,Exterior1st_BrkFace,Exterior1st_CBlock,Exterior1st_CemntBd,Exterior1st_HdBoard,Exterior1st_ImStucc,Exterior1st_MetalSd,Exterior1st_Plywood,Exterior1st_Stone,Exterior1st_Stucco,Exterior1st_VinylSd,Exterior1st_Wd Sdng,Exterior1st_WdShing
Id,,,,,,,,,,,,,,,
2152,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [57]:
complete_dataset.drop(columns = ['Exterior1st_nan'], inplace = True)

Exterior2nd

In [64]:
Exterior2nd_nan_indexes = list(complete_dataset[complete_dataset['Exterior2nd_nan'] == 1].index.values)

In [65]:
cols_to_replace = ['Exterior2nd_AsbShng', 'Exterior2nd_AsphShn', 'Exterior2nd_Brk Cmn', 'Exterior2nd_BrkFace', 'Exterior2nd_CBlock', 'Exterior2nd_CmentBd', 'Exterior2nd_HdBoard',
                   'Exterior2nd_ImStucc', 'Exterior2nd_MetalSd', 'Exterior2nd_Other', 'Exterior2nd_Plywood', 'Exterior2nd_Stone', 'Exterior2nd_Stucco', 'Exterior2nd_VinylSd',
                    'Exterior2nd_Wd Sdng', 'Exterior2nd_Wd Shng']
# Replace values at specified positions with NaN
for row_name in Exterior2nd_nan_indexes:
    row_idx = complete_dataset.index.get_loc(row_name)
    for col_name in cols_to_replace:
        col_idx = complete_dataset.columns.get_loc(col_name)
        complete_dataset.iloc[row_idx, col_idx] = np.nan

In [66]:
complete_dataset[complete_dataset.index.isin(Exterior2nd_nan_indexes)][['Exterior2nd_AsbShng', 'Exterior2nd_AsphShn', 'Exterior2nd_Brk Cmn', 'Exterior2nd_BrkFace',
                                                                                  'Exterior2nd_CBlock', 'Exterior2nd_CmentBd', 'Exterior2nd_HdBoard', 'Exterior2nd_ImStucc', 
                                                                                  'Exterior2nd_MetalSd', 'Exterior2nd_Other', 'Exterior2nd_Plywood', 'Exterior2nd_Stone',
                                                                                  'Exterior2nd_Stucco', 'Exterior2nd_VinylSd', 'Exterior2nd_Wd Sdng', 'Exterior2nd_Wd Shng']]

,Exterior2nd_AsbShng,Exterior2nd_AsphShn,Exterior2nd_Brk Cmn,Exterior2nd_BrkFace,Exterior2nd_CBlock,Exterior2nd_CmentBd,Exterior2nd_HdBoard,Exterior2nd_ImStucc,Exterior2nd_MetalSd,Exterior2nd_Other,Exterior2nd_Plywood,Exterior2nd_Stone,Exterior2nd_Stucco,Exterior2nd_VinylSd,Exterior2nd_Wd Sdng,Exterior2nd_Wd Shng
Id,,,,,,,,,,,,,,,,
2152,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [67]:
complete_dataset.drop(columns = ['Exterior2nd_nan'], inplace = True)

MasVnrType

In [69]:
MasVnrType_nan_indexes = list(complete_dataset[complete_dataset['MasVnrType_nan'] == 1].index.values)

In [70]:
cols_to_replace = ['MasVnrType_BrkCmn', 'MasVnrType_BrkFace', 'MasVnrType_None', 'MasVnrType_Stone']
# Replace values at specified positions with NaN
for row_name in MasVnrType_nan_indexes:
    row_idx = complete_dataset.index.get_loc(row_name)
    for col_name in cols_to_replace:
        col_idx = complete_dataset.columns.get_loc(col_name)
        complete_dataset.iloc[row_idx, col_idx] = np.nan

In [71]:
complete_dataset[complete_dataset.index.isin(MasVnrType_nan_indexes)][['MasVnrType_BrkCmn', 'MasVnrType_BrkFace', 'MasVnrType_None', 'MasVnrType_Stone']]

,MasVnrType_BrkCmn,MasVnrType_BrkFace,MasVnrType_None,MasVnrType_Stone
Id,,,,
2611,NaN,NaN,NaN,NaN


In [72]:
complete_dataset.drop(columns = ['MasVnrType_nan'], inplace = True)

Electrical

In [73]:
Electrical_nan_indexes = list(complete_dataset[complete_dataset['Electrical_nan'] == 1].index.values)

In [74]:
cols_to_replace = ['Electrical_FuseA', 'Electrical_FuseF', 'Electrical_FuseP', 'Electrical_Mix', 'Electrical_SBrkr']
# Replace values at specified positions with NaN
for row_name in Electrical_nan_indexes:
    row_idx = complete_dataset.index.get_loc(row_name)
    for col_name in cols_to_replace:
        col_idx = complete_dataset.columns.get_loc(col_name)
        complete_dataset.iloc[row_idx, col_idx] = np.nan

In [75]:
complete_dataset[complete_dataset.index.isin(Electrical_nan_indexes)][['Electrical_FuseA', 'Electrical_FuseF', 'Electrical_FuseP', 'Electrical_Mix', 'Electrical_SBrkr']]

,Electrical_FuseA,Electrical_FuseF,Electrical_FuseP,Electrical_Mix,Electrical_SBrkr
Id,,,,,
1380,NaN,NaN,NaN,NaN,NaN


In [77]:
complete_dataset.drop(columns = ['Electrical_nan'], inplace = True)

Functional

In [78]:
Functional_nan_indexes = list(complete_dataset[complete_dataset['Functional_nan'] == 1].index.values)

In [79]:
cols_to_replace = ['Functional_Maj1', 'Functional_Maj2', 'Functional_Min1', 'Functional_Min2', 'Functional_Mod', 'Functional_Sev', 'Functional_Typ']
# Replace values at specified positions with NaN
for row_name in Functional_nan_indexes:
    row_idx = complete_dataset.index.get_loc(row_name)
    for col_name in cols_to_replace:
        col_idx = complete_dataset.columns.get_loc(col_name)
        complete_dataset.iloc[row_idx, col_idx] = np.nan

In [80]:
complete_dataset[complete_dataset.index.isin(Functional_nan_indexes)][['Functional_Maj1', 'Functional_Maj2', 'Functional_Min1', 'Functional_Min2', 'Functional_Mod',
                                                                       'Functional_Sev', 'Functional_Typ']]

,Functional_Maj1,Functional_Maj2,Functional_Min1,Functional_Min2,Functional_Mod,Functional_Sev,Functional_Typ
Id,,,,,,,
2217,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2474,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [81]:
complete_dataset.drop(columns = ['Functional_nan'], inplace = True)

SaleType

In [82]:
SaleType_nan_indexes = list(complete_dataset[complete_dataset['SaleType_nan'] == 1].index.values)

In [83]:
cols_to_replace = ['SaleType_COD', 'SaleType_CWD', 'SaleType_Con', 'SaleType_ConLD', 'SaleType_ConLI', 'SaleType_ConLw', 'SaleType_New', 'SaleType_Oth', 'SaleType_WD']
# Replace values at specified positions with NaN
for row_name in SaleType_nan_indexes:
    row_idx = complete_dataset.index.get_loc(row_name)
    for col_name in cols_to_replace:
        col_idx = complete_dataset.columns.get_loc(col_name)
        complete_dataset.iloc[row_idx, col_idx] = np.nan

In [84]:
complete_dataset[complete_dataset.index.isin(SaleType_nan_indexes)][['SaleType_COD', 'SaleType_CWD', 'SaleType_Con', 'SaleType_ConLD', 'SaleType_ConLI', 'SaleType_ConLw',
                                                                     'SaleType_New', 'SaleType_Oth', 'SaleType_WD']]

,SaleType_COD,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD
Id,,,,,,,,,
2490,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [85]:
complete_dataset.drop(columns = ['SaleType_nan'], inplace = True)

In [91]:
complete_dataset

,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,...,BsmtFinType2,HeatingQC,CentralAir,KitchenQual,FireplaceQu,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC
Id,,,,,,,,,,,,,,,,,,,,,
1,65.0,8450.0,7.0,5.0,2003.0,2003.0,196.0,706.0,0.0,150.0,...,1.0,5.0,1.0,4.0,0.0,2.0,3.0,3.0,2.0,0.0
2,80.0,9600.0,6.0,8.0,1976.0,1976.0,0.0,978.0,0.0,284.0,...,1.0,5.0,1.0,3.0,3.0,2.0,3.0,3.0,2.0,0.0
3,68.0,11250.0,7.0,5.0,2001.0,2002.0,162.0,486.0,0.0,434.0,...,1.0,5.0,1.0,4.0,3.0,2.0,3.0,3.0,2.0,0.0
4,60.0,9550.0,7.0,5.0,1915.0,1970.0,0.0,216.0,0.0,540.0,...,1.0,4.0,1.0,4.0,4.0,1.0,3.0,3.0,2.0,0.0
5,84.0,14260.0,8.0,5.0,2000.0,2000.0,350.0,655.0,0.0,490.0,...,1.0,5.0,1.0,4.0,3.0,2.0,3.0,3.0,2.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2915,21.0,1936.0,4.0,7.0,1970.0,1970.0,0.0,0.0,0.0,546.0,...,1.0,4.0,1.0,3.0,0.0,0.0,0.0,0.0,2.0,0.0
2916,21.0,1894.0,4.0,5.0,1970.0,1970.0,0.0,252.0,0.0,294.0,...,1.0,3.0,1.0,3.0,0.0,1.0,3.0,3.0,2.0,0.0
2917,160.0,20000.0,5.0,7.0,1960.0,1996.0,0.0,1224.0,0.0,0.0,...,1.0,5.0,1.0,3.0,3.0,1.0,3.0,3.0,2.0,0.0


# 4. Data Imputation

Now that the dataset has been completely encoded, we can fill the rest of missing values using a k-Nearest Neighbours algorithm. In this case scikit-learn's KNNImputer will be used

In [87]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(complete_dataset.isna().sum())

LotFrontage                 0
LotArea                     0
OverallQual                 0
OverallCond                 0
YearBuilt                   0
YearRemodAdd                0
MasVnrArea                  0
BsmtFinSF1                  0
BsmtFinSF2                  0
BsmtUnfSF                   0
TotalBsmtSF                 0
1stFlrSF                    0
2ndFlrSF                    0
LowQualFinSF                0
GrLivArea                   0
BsmtFullBath                0
BsmtHalfBath                0
FullBath                    0
HalfBath                    0
BedroomAbvGr                0
KitchenAbvGr                0
TotRmsAbvGrd                0
Fireplaces                  0
GarageYrBlt                 0
GarageCars                  1
GarageArea                  1
WoodDeckSF                  0
OpenPorchSF                 0
EnclosedPorch               0
3SsnPorch                   0
ScreenPorch                 0
PoolArea                    0
MiscVal                     0
MoSold    

In [242]:
imputer = KNNImputer(n_neighbors=15)

In [243]:
X = complete_dataset.copy()
X.drop(columns = ['SalePrice'], inplace= True)

In [245]:
tmp = imputer.fit_transform(X)

In [246]:
X = pd.DataFrame(tmp, columns=X.columns, index=X.index)

This data imputation method imputes the average of the values that the nearest neighbours take to the missing values. This averages aren't necessarily integers, so in order to be comply with the one hot encoded variables, this values have to be rounded so that one of the one hot encoded variables related to a categorical variable takes value 1, while the rest of them take 0.

MS_Zoning

In [247]:
X[['MSZoning_C (all)', 'MSZoning_FV', 'MSZoning_RH', 'MSZoning_RL', 'MSZoning_RM']] = X[['MSZoning_C (all)', 'MSZoning_FV', 'MSZoning_RH', 'MSZoning_RL', 'MSZoning_RM']].round()

Utilities

In [248]:
X[['Utilities_AllPub', 'Utilities_NoSeWa']] = X[['Utilities_AllPub', 'Utilities_NoSeWa']].round()

Exterior1st

In [258]:
X.loc[2152, 'Exterior1st_Plywood'] = 1.0

In [260]:
Exterior1st_columns = ['Exterior1st_AsbShng', 'Exterior1st_AsphShn', 'Exterior1st_BrkComm', 'Exterior1st_BrkFace', 'Exterior1st_CBlock', 'Exterior1st_CemntBd', 'Exterior1st_HdBoard',
                       'Exterior1st_ImStucc', 'Exterior1st_MetalSd', 'Exterior1st_Plywood', 'Exterior1st_Stone', 'Exterior1st_Stucco', 'Exterior1st_VinylSd', 'Exterior1st_Wd Sdng',
                       'Exterior1st_WdShing']

In [262]:
X[Exterior1st_columns] = X[Exterior1st_columns].round()

In [376]:
#Check that the variables have been correctly rounded
(X[Exterior1st_columns].sum(axis = 1) == 1).sum()

2919

Exterior2nd

In [288]:
X.loc[2152, 'Exterior2nd_Plywood'] = 1.0

In [290]:
Exterior2nd_columns = ['Exterior2nd_AsbShng', 'Exterior2nd_AsphShn', 'Exterior2nd_Brk Cmn', 'Exterior2nd_BrkFace', 'Exterior2nd_CBlock', 'Exterior2nd_CmentBd', 'Exterior2nd_HdBoard',
                       'Exterior2nd_ImStucc', 'Exterior2nd_MetalSd', 'Exterior2nd_Other', 'Exterior2nd_Plywood', 'Exterior2nd_Stone', 'Exterior2nd_Stucco', 'Exterior2nd_VinylSd',
                       'Exterior2nd_Wd Sdng', 'Exterior2nd_Wd Shng']

In [291]:
X[Exterior2nd_columns] = X[Exterior2nd_columns].round()

In [293]:
#Check that the variables have been correctly rounded
(X[Exterior2nd_columns].sum(axis = 1) == 1).sum()

2919

MasVnrType

In [299]:
X[['MasVnrType_BrkCmn', 'MasVnrType_BrkFace', 'MasVnrType_None', 'MasVnrType_Stone']] = X[['MasVnrType_BrkCmn', 'MasVnrType_BrkFace', 'MasVnrType_None', 'MasVnrType_Stone']].round()

In [306]:
#Check that the variables have been correctly rounded
(X[['MasVnrType_BrkCmn', 'MasVnrType_BrkFace', 'MasVnrType_None', 'MasVnrType_Stone']].sum(axis = 1) == 1).sum()

2919

Electrical

In [313]:
X[['Electrical_FuseA', 'Electrical_FuseF', 'Electrical_FuseP', 'Electrical_Mix', 'Electrical_SBrkr']] = X[['Electrical_FuseA', 'Electrical_FuseF', 'Electrical_FuseP',
                                                                                                            'Electrical_Mix', 'Electrical_SBrkr']] .round()

In [319]:
#Check that the variables have been correctly rounded
(X[['Electrical_FuseA', 'Electrical_FuseF', 'Electrical_FuseP', 'Electrical_Mix', 'Electrical_SBrkr']].sum(axis = 1) == 1).sum()

2919

Functional

In [330]:
Functional_columns = ['Functional_Maj1', 'Functional_Maj2', 'Functional_Min1', 'Functional_Min2', 'Functional_Mod', 'Functional_Sev', 'Functional_Typ']

In [331]:
X[Functional_columns] = X[Functional_columns].round()

In [334]:
#Check that the variables have been correctly rounded
(X[Functional_columns].sum(axis = 1) == 1).sum()

2919

Saletype

In [335]:
SaleType_columns = ['SaleType_COD', 'SaleType_CWD', 'SaleType_Con', 'SaleType_ConLD', 'SaleType_ConLI', 'SaleType_ConLw', 'SaleType_New', 'SaleType_Oth', 'SaleType_WD']

In [346]:
X[SaleType_columns] = X[SaleType_columns].round()

In [349]:
#Check that the variables have been correctly rounded
(X[SaleType_columns].sum(axis = 1) == 1).sum()

2919

# 5. Merge dataset

The final dataset with all the encoded and imputed data is created and split into 2, the one with the training points (encoded_train_dataset) and one with the test points (encoded_test_dataset).

In [361]:
final_dataset = pd.concat([X, complete_dataset['SalePrice']], axis = 1)

In [362]:
final_dataset

,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,...,HeatingQC,CentralAir,KitchenQual,FireplaceQu,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,65.0,8450.0,7.0,5.0,2003.0,2003.0,196.0,706.0,0.0,150.0,...,5.0,1.0,4.0,0.0,2.0,3.0,3.0,2.0,0.0,208500.0
2,80.0,9600.0,6.0,8.0,1976.0,1976.0,0.0,978.0,0.0,284.0,...,5.0,1.0,3.0,3.0,2.0,3.0,3.0,2.0,0.0,181500.0
3,68.0,11250.0,7.0,5.0,2001.0,2002.0,162.0,486.0,0.0,434.0,...,5.0,1.0,4.0,3.0,2.0,3.0,3.0,2.0,0.0,223500.0
4,60.0,9550.0,7.0,5.0,1915.0,1970.0,0.0,216.0,0.0,540.0,...,4.0,1.0,4.0,4.0,1.0,3.0,3.0,2.0,0.0,140000.0
5,84.0,14260.0,8.0,5.0,2000.0,2000.0,350.0,655.0,0.0,490.0,...,5.0,1.0,4.0,3.0,2.0,3.0,3.0,2.0,0.0,250000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2915,21.0,1936.0,4.0,7.0,1970.0,1970.0,0.0,0.0,0.0,546.0,...,4.0,1.0,3.0,0.0,0.0,0.0,0.0,2.0,0.0,NaN
2916,21.0,1894.0,4.0,5.0,1970.0,1970.0,0.0,252.0,0.0,294.0,...,3.0,1.0,3.0,0.0,1.0,3.0,3.0,2.0,0.0,NaN
2917,160.0,20000.0,5.0,7.0,1960.0,1996.0,0.0,1224.0,0.0,0.0,...,5.0,1.0,3.0,3.0,1.0,3.0,3.0,2.0,0.0,NaN


In [367]:
encoded_train_dataset = final_dataset.iloc[0:1460, :]

In [371]:
encoded_test_dataset = final_dataset.iloc[1460:, :-1]

# 6. Save datasets to csv files

In [374]:
encoded_train_dataset.to_csv('encoded_train_dataset.csv', sep = ',')

In [375]:
encoded_test_dataset.to_csv('encoded_test_dataset.csv', sep = ',')